### 照片拍攝，透過opencv拍照

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

num = 0

while cap.isOpened():

    succes, img = cap.read()

    k = cv2.waitKey(5)

    if k == ord("q"):
        break
    elif k == ord('s'): # wait for 's' key to save and exit
        cv2.imwrite('images/img' + str(num) + '.png', img)
        print("image saved!")
        num += 1

    cv2.imshow('Img',img)

cap.release()

cv2.destroyAllWindows()

### 繪圖函數宣告 一定要先執行！！

In [ ]:
# 繪製函數定義
import numpy as np
def draw_axis(img, corners, image_points):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(np.int16(image_points[0].ravel())), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[1].ravel())), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(np.int16(image_points[2].ravel())), (0,0,255), 5)
    return img
def draw_xy_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[0].ravel())), tuple(np.int16(image_points[1].ravel())), (0,0,0), 5)
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[2].ravel())), (0,0,0), 5)
    return img
def draw_z_lines(img, image_points):
    img = cv2.line(img, tuple(np.int16(image_points[1].ravel())), tuple(np.int16(image_points[3].ravel())), (128,128,0), 5)
    return img

# 鏡頭校正

In [ ]:
import numpy as np
import cv2
import glob
import pickle

# 定義所使用的棋盤格格式
chessboardSize = (9,6)
# 亞像素計算的迭代終止條件
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# 棋盤座標矩陣序列矩陣創立
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

# 棋盤格單格在真實圖片中的寬度大小
size_of_chessboard_squares_mm = 19
objp = objp * size_of_chessboard_squares_mm

#分別創建用於儲存真實座標與對應像素座標的陣列，陣列中的每個元素代表的是每張圖對應到一組棋盤座標與圖片像素座標
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# 抓取指定路徑所有檔案路徑，輸出為list[str]
images = glob.glob('your/images/location/*.png')
# images = glob.glob('camera/images/*.png')

for result_img in images:

    img = cv2.imread(result_img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 尋找棋盤格角點
    ret, corners = cv2.findChessboardCorners(gray, chessboardSize, None)

    # 若尋找成功就將對應的棋盤座標點與像素點新增近objpoints和imgpoints list當中
    if ret == True:

        objpoints.append(objp)
        # 將找到的圖像像素座標點進行亞像素修正處理，以取得更精確的像素座標位置
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        cv2.drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(1000)


cv2.destroyAllWindows()


# 使用讀取到的圖片座標點與對應的棋盤座標進行鏡頭校正
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# 儲存鏡頭內參成pkl檔，以便之後調用，而無需重新計算鏡頭內參矩陣與畸變係數
pickle.dump((cameraMatrix, dist), open( "calibration.pkl", "wb" ))

# 桌面座標系建立

### 視窗建立函數

In [ ]:
import tkinter as tk

# 創建獲取長寬表單視窗
def setHeightWin():
    root = tk.Tk()
    root.title("Input Form")

    # 獲取螢幕寬高
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    # 設置視窗寬高(像素)
    window_width = 400  
    window_height = 150 

    # 計算視窗左上角座標
    x = (screen_width - window_width) // 2
    y = (screen_height - window_height) // 2

    # 視窗生成
    root.geometry(f"{window_width}x{window_height}+{x}+{y}")

    # 創建label, entry元件顯示讀取參數
    label_height = tk.Label(root, text="Height:", font=("Helvetica", 14))
    label_height.pack()

    entry_height = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_height.pack()

    label_width = tk.Label(root, text="Width:", font=("Helvetica", 14))
    label_width.pack()

    entry_width = tk.Entry(root, font=("Helvetica", 14),justify="center")
    entry_width.pack()
    
    result = []
    # 處理送出的表單
    def submit_form():
        height = entry_height.get()
        width = entry_width.get()
        result.extend([height, width])

        if any(item == "" for item in result):# 當缺少參數的時候就將參數清空，不採取動作
            result.clear()
            return
        # 參數沒問題就銷毀視窗，回傳數值
        root.destroy()
    # 將enter綁定送出表單
    root.bind('<Return>', lambda event=None: submit_form())
    # 創建"送出"按鈕
    submit_button = tk.Button(root, text="送出", command=submit_form,width=10,height=1,font=("Helveticsa",14))
    submit_button.pack()

    # 啟動Tkinter主循環
    root.mainloop()
    return result

### 主程式

In [ ]:
import cv2

#讀取寬高參數
square_height,square_width = setHeightWin()
square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])

with open('camera/calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)

image = cv2.imread('pnp_test_img/img101.png')  # 載入圖片
height =480
scale = height / image.shape[0]
image = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
drawing_image = image.copy()
Adjusted_image = None 

isDrawing = True
isAdjusting = False
isFirstDraw = True
square_image_points = []
line_thickness = 2
# 回調函數，處理滑鼠事件
def draw_line(event, x, y, flags, param):
    global line_start, line_end, isDrawing,drawing_image,Adjusted_image,square_image_points,isAdjusting,isFirstDraw,line_thickness
    #偵測事件
    if event == cv2.EVENT_LBUTTONDOWN:  # 當按下左鍵
        if isDrawing:
            if isFirstDraw:
                isFirstDraw = False
            else :
                cv2.line(drawing_image,square_image_points[-1],(x,y),(255,0,0),line_thickness)
            if len(square_image_points) < 4:
                square_image_points.append((x,y))
            if len(square_image_points)==4:
                isDrawing = False
                isAdjusting = True
                frame_update()
        elif isAdjusting:
            distances = np.linalg.norm(np.array(square_image_points) - np.array((x,y)), axis=1)
            nearest_index = np.argmin(distances)
            square_image_points[nearest_index]=(x,y)
            frame_update()
    elif event == cv2.EVENT_MOUSEMOVE:  # 當滑鼠移動
        if isDrawing and not isFirstDraw:
            temp_image = drawing_image.copy()  # 複製圖像而非變更原圖
            cv2.line(temp_image, square_image_points[-1], (x, y), (0, 255, 0), line_thickness)
            cv2.imshow('Set Table axis', temp_image)
        if isAdjusting:
            cv2.imshow('Set Table axis', Adjusted_image)

    elif event == cv2.EVENT_MOUSEWHEEL: # 當滑鼠滾輪滾動
        delta = flags
        if delta > 0:
            if line_thickness==3:
                pass
            else:
                line_thickness+=1
        elif delta < 0:
            if line_thickness == 1:
                pass
            else:
                line_thickness-=1
        frame_update()

# 創建視窗
cv2.namedWindow('Set Table axis')
# 綁定監聽滑鼠事件，綁定處理事件回調函數
cv2.setMouseCallback('Set Table axis', draw_line)

def frame_update(): # 進行畫面更新
    global Adjusted_image, square_image_points,line_thickness,rvecs,tvecs
    Adjusted_image=image.copy()
    cv2.polylines(Adjusted_image, [np.array(square_image_points)], isClosed=True, color=(0, 255, 0), thickness=line_thickness)
    for coordinate,point in zip(square_object_points,square_image_points):
        cv2.putText(Adjusted_image, f"{list(coordinate)}", point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0, 255), 2)
    square_img_points = np.float32(square_image_points)
    # print(square_img_points)
    ret,rvecs,tvecs = cv2.solvePnP(square_object_points,square_img_points,mtx,dist)
    # print(rvecs)
    axis = np.float32([[100,0,0], [0,100,0], [0,0,100]]).reshape(-1,3)
    axis_img_points, _ = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    draw_axis(Adjusted_image,square_img_points.astype(int),axis_img_points)

# 顯示圖像
while True:
    if not isAdjusting:
        cv2.imshow('Set Table axis', drawing_image)
    else:
        cv2.imshow('Set Table axis', Adjusted_image)
    key = cv2.waitKey(1)
    if key == 27:  # 當按下ESC退出循環
        break
    if len(square_image_points)!=4: #確認是否四個像素座標點抓到了
        continue
    if key == ord("r"): # Z軸對調
        square_image_points[1],square_image_points[3]=square_image_points[3],square_image_points[1]
        # square_coordinates[1],square_coordinates[3]=square_coordinates[3],square_coordinates[1]
        frame_update()
    if key == ord("e"): # 變更原點
        square_image_points=square_image_points[-1:]+ square_image_points[:-1] #前包後不包
        frame_update()
    if key == ord("w"): # 重設長寬
        square_height,square_width = setHeightWin()
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key ==ord("q"): # 長寬互換
        square_height,square_width=square_width,square_height
        square_object_points=np.float32([(0,0,0),(square_width,0,0),(square_width,square_height,0),(0,square_height,0)])
        frame_update()
    if key == ord("s"): # 儲存真實與圖像座標和PNP計算結果
        with open('table_points.pkl', 'wb') as file:
                pickle.dump((square_object_points,np.float32(square_image_points)), file)
        with open('table_solvePnP.pkl', 'wb') as file:
                pickle.dump((rvecs,tvecs), file)
        print("saved")
                    
cv2.destroyAllWindows()

# 方塊座標系轉換至桌面座標系

模型讀取

In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import cube_detector.cube_detector as CD
import pickle

model = YOLO("yolov8n-seg-custom.pt")
surface_model = YOLO('cube_surface_seg2.pt')


方塊到桌面座標系轉換

### 照片計算

In [ ]:
detector=CD.CubeDetector(model,surface_model)
detector
with open('test_calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('test_table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('test_table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
color = "green"
img = cv2.imread("test.jpg")
height =480
scale = height / img.shape[0]
img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
#  將圖片丟入偵測器進行偵測
result_img = detector.detect(img,index=None,color=None)
# 讀取抓到的角點座標
cube_imagePoints=detector.get_cube_sequence_imagePoints(color)
# 依順序定出對應真實世界對應座標
cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0),(25,0,-25),(0,0,-25)])
# 如果抓到六個點採用EPNP算法(較準確穩定)，四個點則是一般算法
if cube_imagePoints.shape[0]==6:
    PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist,flags=cv2.SOLVEPNP_EPNP)
elif cube_imagePoints.shape[0]==4:
    PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints[:4].astype(float),cube_imagePoints,mtx,dist)
#  若成功計算出來則計算出該方塊相對於桌面座標系的座標位置
if PnP_success:
    R1, _ = cv2.Rodrigues(cube_rvec) # 創建方塊到相機座標系的旋轉矩陣
    R2, _ = cv2.Rodrigues(table_rvec) # 創建桌面到方塊座標系的旋轉矩陣
    T1 = np.eye(4) #創建4*4單位矩陣
    T2 = np.eye(4) 
    T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T #  方塊座標系旋轉加平移到相機座標系 轉換矩陣
    T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系 轉換矩陣
    T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
    transform_matrix = T2 @ T1 # 結合二者，得到方塊座標系到桌面座標系的轉換矩陣
    transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T # 傳入方塊正中央座標，即可得到方塊在桌面座標位置了
    # 输出结果
    print("物體在桌面座標系下的座標:\n", transformed_point)
# xyz座標位置線繪製
x,y,z = transformed_point[0:3].flatten()
xy_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0],[x,y,z]]).reshape(-1,3)
xy_line_points_img,_=cv2.projectPoints(xy_line_points,table_rvec,table_tvec,mtx,dist)
result_img = draw_xy_lines(result_img,xy_line_points_img)
result_img = draw_z_lines(result_img,xy_line_points_img)

# xyz座標軸繪製
axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
result_img=draw_axis(result_img,cube_imagePoints.astype(int),axis_cube_img_points)
result_img=draw_axis(result_img,table_imagePoints.astype(int),axis_table_img_points)

# 取得方塊輪廓計算方塊重心位置
contour_outer = detector.get_cube_contour_outer(color)
radius = int(0.04*cv2.arcLength(contour_outer,True))
print(radius)
M = cv2.moments(contour_outer)
if M["m00"] != 0:
    centroid_X = int(M["m10"] / M["m00"])# 算形心x
    centroid_Y = int(M["m01"] / M["m00"])# 算形心y
# 繪製重心座標點
result_img = cv2.circle(result_img, (centroid_X,centroid_Y),radius, (255,0,255), 2)

# 繪製PNP計算所得座標點
predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x,y,z]),table_rvec,table_tvec,mtx,dist)
predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)

cv2.putText(result_img, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 128), 2)

# 推算是否位置估計錯誤
if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)>radius:
    cv2.putText(result_img, f"Error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
    result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,0,255), -1)
else:
    cv2.putText(result_img, f"Ok", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,255,0), -1)
cv2.imshow("result",result_img)
cv2.waitKey(0)

    

### 即時偵測主程式
鏡頭開啟很費時

In [ ]:
cap = cv2.VideoCapture(0)

，所以真的要關閉再手動關閉

In [ ]:
cap.release()

In [ ]:
import importlib


importlib.reload(CD)

In [ ]:
detector=CD.CubeDetector(model,surface_model)

with open('test_calibration.pkl', 'rb') as file:
    mtx,dist = pickle.load(file)
with open('test_table_points.pkl','rb') as file:
    table_objectPoints,table_imagePoints = pickle.load(file)
with open('test_table_solvePnP.pkl','rb') as file:
    table_rvec,table_tvec = pickle.load(file)
color = "green"
while cap.isOpened():
    cap_success, frame = cap.read()
    if cap_success:
        height =480
        scale = height / frame.shape[0]
        frame = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        #  將圖片丟入偵測器進行偵測
        result_img = detector.detect(frame,index=None,color=None)
        # 讀取抓到的角點座標
        cube_imagePoints=detector.get_cube_sequence_imagePoints(color)
        if cube_imagePoints is None:
            cv2.imshow("result",frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            continue
        # 依順序定出對應真實世界對應座標
        cube_objectPoints=np.array([(0,0,0),(0,25,0),(25,25,0),(25,0,0),(25,0,-25),(0,0,-25)])
        # 如果抓到六個點採用EPNP算法(較準確穩定)，四個點則是一般算法
        if cube_imagePoints.shape[0]==6:
            PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints.astype(float),cube_imagePoints,mtx,dist,flags=cv2.SOLVEPNP_EPNP)
        elif cube_imagePoints.shape[0]==4:
            PnP_success,cube_rvec,cube_tvec = cv2.solvePnP(cube_objectPoints[:4].astype(float),cube_imagePoints,mtx,dist)
        #  若成功計算出來則計算出該方塊相對於桌面座標系的座標位置
        if PnP_success:
            R1, _ = cv2.Rodrigues(cube_rvec) # 創建方塊到相機座標系的旋轉矩陣
            R2, _ = cv2.Rodrigues(table_rvec) # 創建桌面到方塊座標系的旋轉矩陣
            T1 = np.eye(4) #創建4*4單位矩陣
            T2 = np.eye(4) 
            T1[:3, :3], T1[:3, 3] = R1, cube_tvec.T #  方塊座標系旋轉加平移到相機座標系 轉換矩陣
            T2[:3, :3], T2[:3, 3] = R2, table_tvec.T # 桌面座標系旋轉加平移到相機座標系 轉換矩陣
            T2=np.linalg.inv(T2) # 取逆矩陣，變成 相譏座標系到桌面座標系 轉換矩陣
            transform_matrix = T2 @ T1 # 結合二者，得到方塊座標系到桌面座標系的轉換矩陣
            transformed_point = transform_matrix @ np.array([[12.5, 12.5, -12.5, 1]]).T # 傳入方塊正中央座標，即可得到方塊在桌面座標位置了
            # 输出结果
            print("物體在桌面座標系下的座標:\n", transformed_point)
        # xyz座標位置線繪製
        x,y,z = transformed_point[0:3].flatten()
        xy_line_points=np.float32([[0,y,0],[x,y,0],[x,0,0],[x,y,z]]).reshape(-1,3)
        xy_line_points_img,_=cv2.projectPoints(xy_line_points,table_rvec,table_tvec,mtx,dist)
        result_img = draw_xy_lines(result_img,xy_line_points_img)
        result_img = draw_z_lines(result_img,xy_line_points_img)

        # xyz座標軸繪製
        axis = np.float32([[25,0,0], [0,25,0], [0,0,25]]).reshape(-1,3)
        axis_cube_img_points, _ = cv2.projectPoints(axis, cube_rvec, cube_tvec, mtx, dist)
        axis_table_img_points, _ = cv2.projectPoints(axis, table_rvec, table_tvec, mtx, dist)
        result_img=draw_axis(result_img,cube_imagePoints.astype(int),axis_cube_img_points)
        result_img=draw_axis(result_img,table_imagePoints.astype(int),axis_table_img_points)

        # 取得方塊輪廓計算方塊重心位置
        contour_outer = detector.get_cube_contour_outer(color)
        radius = int(0.04*cv2.arcLength(contour_outer,True))
        print(radius)
        M = cv2.moments(contour_outer)
        if M["m00"] != 0:
            centroid_X = int(M["m10"] / M["m00"])# 算形心x
            centroid_Y = int(M["m01"] / M["m00"])# 算形心y
        # 繪製重心座標點
        result_img = cv2.circle(result_img, (centroid_X,centroid_Y),radius, (255,0,255), 2)

        # 繪製PNP計算所得座標點
        predict_cube_center_imagePoint = cv2.projectPoints(np.float32([x,y,z]),table_rvec,table_tvec,mtx,dist)
        predict_cube_center_imagePoint = predict_cube_center_imagePoint[0].astype(np.int16)
        
        cv2.putText(result_img, f"{x=:.1f}, {y=:.1f}, {z=:.1f}", (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (128, 0, 128), 2)

        # 推算是否位置估計錯誤
        if np.linalg.norm(np.array((centroid_X,centroid_Y))-predict_cube_center_imagePoint)>radius:
            cv2.putText(result_img, f"Error", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
            result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,0,255), -1)
        else:
            cv2.putText(result_img, f"Ok", (10,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
            result_img = cv2.circle(result_img, predict_cube_center_imagePoint.ravel(),5, (0,255,0), -1)
        cv2.imshow("result",result_img)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else :
        break
cv2.destroyAllWindows()